In [156]:
import json
import pandas as pd
import requests
import os
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [157]:
#set the path to open the json configuration for the database
#__location__ = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname(__file__)))
#import database password and api key
#with open(os.path.join(__location__,'api_key.json')) as f:
#    config = json.load(f)

In [158]:
with open('api_key', 'r') as f:
    config = json.load(f)
key = config['eia_API']['api_key']

password = config['eia_API']['password']
user_name = config['eia_API']['user_name']
database = config['eia_API']['database']
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE="+str(database)+";UID="+str(user_name)+";PWD="+str(password)

params = urllib.parse.quote_plus(connection_string)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection=engine.connect()

In [179]:
def add_key(url,key=key):
    url = url.replace('YOUR_API_KEY_HERE',str(key))
    return(url)

In [382]:
def return_json(category_id,category):
    if category:
        url = 'http://api.eia.gov/category/?api_key=YOUR_API_KEY_HERE&category_id=CATEGORY'
    else:
        url = 'http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=CATEGORY'
    url = add_key(url)
    url = url.replace('CATEGORY',str(category_id))
    r = requests.get(url, allow_redirects=True, stream=True, headers=headers).json()
    #r = r['category']
    parent = r['category']['parent_category_id']
    children = r['category']['childcategories']
    df = pd.DataFrame(children)
    df['Parent_ID'] = parent
    df['parent'] = category_id
    return(df)

In [186]:
eia_url = 'http://api.eia.gov/category/?api_key=YOUR_API_KEY_HERE&category_id=371'
spot_prices = 'http://api.eia.gov/category/?api_key=YOUR_API_KEY_HERE&category_id=714757'
eia_url = add_key(eia_url,key)

In [187]:
#get top level

In [236]:
# gather neccecary codes

In [275]:
crude_prices = ['PET.RWTC.D','PET.RBRTE.D']

In [300]:
def return_df(js):
    series_id = js['request']['series_id']
    name=js['series'][0]['name']
    units = js['series'][0]['units']
    frequency = js['series'][0]['f']
    units_abbrev = js['series'][0]['unitsshort']
    source = js['series'][0]['source']
    updates = js['series'][0]['updated']
    description = js['series'][0]['description']
    df = pd.DataFrame(js['series'][0]['data'])
    df.rename(columns = {0:'Date',1:'Value'},inplace=True)
    df['Data'] = description
    df['Frequency'] = frequency
    df['Units'] = units_abbrev
    df['Units Description'] = units
    #todo: add option to return a metadataframe
    df['series id'] = series_id
    return(df)

In [309]:
def gather_df(code_list):
    df_list = []
    for code in code_list:
        js = return_json(str(code),False)
        df = return_df(js)
        df_list.append(df)
    df = pd.concat(df_list, axis=0, sort=False, ignore_index=True)
    return(df)

In [310]:
df = gather_df(crude_prices)

In [312]:
df.head()

,Date,Value,Data,Frequency,Units,Units Description,series id
0,20190225,55.32,"Cushing, OK WTI Spot Price FOB",D,$/bbl,Dollars per Barrel,PET.RWTC.D
1,20190222,57.01,"Cushing, OK WTI Spot Price FOB",D,$/bbl,Dollars per Barrel,PET.RWTC.D
2,20190221,56.95,"Cushing, OK WTI Spot Price FOB",D,$/bbl,Dollars per Barrel,PET.RWTC.D
3,20190220,56.90,"Cushing, OK WTI Spot Price FOB",D,$/bbl,Dollars per Barrel,PET.RWTC.D
4,20190219,56.12,"Cushing, OK WTI Spot Price FOB",D,$/bbl,Dollars per Barrel,PET.RWTC.D


In [400]:
first = return_json(371,True)
first

,category_id,name,Parent_ID,parent
0,0,Electricity,None,371
1,40203,State Energy Data System (SEDS),None,371
2,711224,Total Energy,None,371
3,714755,Petroleum,None,371
4,714804,Natural Gas,None,371
5,717234,Coal,None,371
6,829714,Short-Term Energy Outlook,None,371
7,964164,Annual Energy Outlook,None,371
8,1292190,Crude Oil Imports,None,371
9,2123635,U.S. Electric System Operating Data,None,371


In [394]:
second = return_json(0,True)
second.head()

,category_id,name,Parent_ID,parent
0,1,Net generation,371,0
1,32,Total consumption (Btu),371,0
2,33,Consumption for electricity generation (Btu),371,0
3,34,Consumption for useful thermal output (Btu),371,0
4,35,Total consumption,371,0


In [398]:
def gather_children(df):
    lis = []
    for child in df['category_id']:
        child_df = return_json(int(child),True)
        lis.append(child_df)
    child = pd.concat(lis, axis=0, sort=False, ignore_index=True)
    return(child)

In [399]:
nex = gather_children(first)
nex

,category_id,name,Parent_ID,parent
0,1,Net generation,371,0
1,32,Total consumption (Btu),371,0
2,33,Consumption for electricity generation (Btu),371,0
3,34,Consumption for useful thermal output (Btu),371,0
4,35,Total consumption,371,0
5,36,Consumption for electricity generation,371,0
6,37,Consumption for useful thermal output,371,0
7,38,Retail sales of electricity,371,0
8,39,Revenue from retail sales of electricity,371,0
9,40,Average retail price of electricity,371,0


In [395]:
def merger(l,r,left_name):
    master = l.merge(r,how = 'left', left_on='category_id',right_on='parent',suffixes=('',str(left_name)))
    return(master)

In [397]:
master = merger(first,second,'_child')
master.head()

,category_id,name,Parent_ID,parent,category_id_child,name_child,Parent_ID_child,parent_child
0,0,Electricity,None,371,1.0,Net generation,371,0.0
1,0,Electricity,None,371,32.0,Total consumption (Btu),371,0.0
2,0,Electricity,None,371,33.0,Consumption for electricity generation (Btu),371,0.0
3,0,Electricity,None,371,34.0,Consumption for useful thermal output (Btu),371,0.0
4,0,Electricity,None,371,35.0,Total consumption,371,0.0
